## Setup

In [ ]:
!pip uninstall -qqy jupyterlab kfp
!pip install -q "google-genai==1.7.0" "pypdf" "python-docx" "requests"

In [ ]:
import os
import google.generativeai as genai
from google.colab import files
from pypdf import PdfReader
from docx import Document
from IPython.display import display, Markdown, HTML
import re
import pandas as pd
from google.generativeai import types
from datetime import datetime, timedelta

In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)

## CV Refinement Agent

In [ ]:
class CVRefinementAgent:
    def __init__(self, model_name="gemini-1.5-pro"):
        self.model = genai.GenerativeModel(model_name)

    def upload_cv(self):
        uploaded = files.upload()
        self.file_name = next(iter(uploaded.keys()))
        return self.file_name

    def extract_text(self, file_path):
        if file_path.lower().endswith(".pdf"):
            reader = PdfReader(file_path)
            return "\n".join(page.extract_text() for page in reader.pages if page.extract_text())
        elif file_path.lower().endswith(".docx"):
            doc = Document(file_path)
            return "\n".join(p.text for p in doc.paragraphs if p.text.strip())
        else:
            raise ValueError("Unsupported file format")

    def build_prompt(self, cv_text):
        return f"""
You are a senior recruiter and career coach reviewing a candidate’s CV.

Your task has two parts:

1. **Critically review** each section of the CV. For each section (e.g., Name, Contact, Education, Experience, Skills, Projects, Certifications, etc.), do the following:
   - Identify if it exists or is missing.
   - Provide comments and suggestions for improvements.
   - Point out outdated language, formatting issues, unclear points, or lack of impact.
   - Suggest how to improve with clarity, action verbs, keywords, or structure.

2. **Write a fully updated and improved version** of the CV based on your suggestions.
   - Maintain the original section order unless you suggest otherwise.
   - Keep the tone professional, use bullet points when appropriate.
   - Modernize formatting and phrasing.
   - Avoid personal pronouns ("I", "my").
   - Keep all original info, just refine.

Return your response in **two clear parts**:

---

### 1. COMMENTS AND SUGGESTIONS
Write your suggestions here.

---

### 2. REWRITTEN CV
Write the improved version here.

---

Here is the candidate’s CV:

{cv_text}
"""

    def run_review(self):
        print("📂 Please upload your CV (PDF or DOCX)...")
        file_path = self.upload_cv()
        print(f"✅ File '{file_path}' uploaded.")

        print("🔍 Extracting text...")
        cv_text = self.extract_text(file_path)

        print("🧠 Sending to Gemini model for review and rewrite...")
        prompt = self.build_prompt(cv_text)
        response = self.model.generate_content(prompt)

        print("✅ Done!\n")
        return response.text, cv_text

### Run

In [ ]:
agent = CVRefinementAgent()
output, cv_text = agent.run_review()
Markdown(output)

## Job Search Agent

In [ ]:
from google import genai
from google.genai import types
from IPython.display import Markdown
import textwrap

In [ ]:
class JobSearchAgent:
    def __init__(self, api_key, model_name="gemini-2.0-flash"):
        self.client = genai.Client(api_key=api_key)
        self.model_name = model_name
        self.chat = None

    def start_chat(self):
        config_with_search = types.GenerateContentConfig(
            tools=[types.Tool(google_search=types.GoogleSearch())],
            temperature=0.3
        )
        self.chat = self.client.chats.create(
            model=self.model_name,
            config=config_with_search
        )

    def ask_job_filters(self):
        print("Please enter your job preferences. Press enter to skip any.")
        location = input("📍 Preferred location: ").strip()
        job_level = input("🎓 Desired level (e.g., intern, junior, senior): ").strip()
        job_type = input("🕒 Full-time / Part-time / Remote: ").strip()
        return location, job_level, job_type

    def build_search_prompt(self, cv_text, location, level, job_type):
      today = datetime.today()
      one_month_ago = today - timedelta(days=30)
      today_str = today.strftime("%B %d, %Y")  # e.g., "April 18, 2025"
      one_month_ago_str = one_month_ago.strftime("%B %d, %Y")

      base_prompt = f"""
  You are an expert AI recruiter assistant with access to up-to-date job listings via Google Search.

  First, based on the user's resume, identify the field of expertise, target industries, and relevant job titles.
Then, create a list of reputable companies that are active in this domain (e.g., via Google Search like "top companies in aerospace AI", "leading robotics firms", etc.).

For each identified company, go to their official careers page (e.g. companyname.com/careers) and look for job openings relevant to the user’s background.
  ---

   **Candidate CV Summary**
  Below is the resume content of the candidate. Use it to understand the candidate’s background, expertise, skills, and professional goals:

  {cv_text}

  ---

   **Time Filter (IMPORTANT)**
  Read the job post and only include job postings published or updated **between {one_month_ago_str} and {today_str}** Or the deadline is stillreachable.
  Exclude expired or outdated job listings. Filter out results older than one month.

  ---

   **Job Matching Criteria**
  - Only return jobs that are relevant to the candidate’s background.
  - Return **as many jobs as you can** that is high relevant.
  - Evaluate compatibility based on the candidate's resume, required skills, responsibilities, location, and experience level.
  - The job position should be still open and update

   **User Preferences**
  - Location: {location or "no preference (remote or any location allowed)"}
  - Experience Level: {level or "any level"}
  - Job Type: {job_type or "full flexibility (any type)"}

  ---

   **Format of Each Job Result**
  Return a list of job results. Each one must contain:
  (number of the result) **Job Title**
      - **Company Name**
      - **Location**
      - **Short Description** (max 3 lines)
      - **Job URL or Application Link**
      - **Estimated Compatibility Score**
      - **Reason for Match** (1–2 lines why this job is a good fit)

  Only include jobs that have enough detail and meet the time and match criteria.

  At the end, write a short summary with:
  - Number of matched jobs
  - What made them compatible
  """
      return base_prompt

    def search_jobs(self, cv_text):
        self.start_chat()
        location, level, job_type = self.ask_job_filters()
        query = self.build_search_prompt(cv_text, location, level, job_type)
        response = self.chat.send_message(query)
        return response

    def show_results(self, response):
        print("\n🔎 Matched Jobs:\n")
        for part in response.candidates[0].content.parts:
            if part.text:
                display(Markdown(textwrap.dedent(part.text)))

### Run

In [ ]:
job_agent = JobSearchAgent(api_key=GOOGLE_API_KEY)
response = job_agent.search_jobs(cv_text)
job_agent.show_results(response)

## CV Customizer Agent

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
class CVCustomizerAgent:
    def __init__(self, api_key, cv_text, model_name="gemini-1.5-pro"):
        self.cv_text = cv_text
        self.client = genai.Client(api_key=api_key)
        self.model_name = model_name
        self.chat = None
        self.session = requests.Session()
        self.session.headers.update({"User-Agent": "Mozilla/5.0"})

    def extract_job_description(self, url):
        try:
            print("🌐 Scraping job description from the URL...")
            resp = self.session.get(url, timeout=10)
            soup = BeautifulSoup(resp.text, "html.parser")

            tags = soup.find_all(["p", "li", "div", "section", "article"])
            job_text = "\n".join([tag.get_text(strip=True) for tag in tags if len(tag.get_text(strip=True)) > 40])
            return job_text
        except Exception as e:
            print("❌ Error extracting job description:", e)
            return ""

    def start_chat(self):
        self.chat = self.client.chats.create(
            model=self.model_name,
            config=types.GenerateContentConfig(temperature=0.3)
        )

    def build_prompt(self, job_description):
        return f"""
You are a professional recruiter and expert resume customizer.

Your task has **two parts**:

---

##  1. ANALYZE THE JOB DESCRIPTION

- Identify the job title, required qualifications, responsibilities, and essential skills.
- Understand what the employer is really looking for (e.g., technical expertise, leadership, adaptability, etc.).
- Determine the tone and style of the job post (e.g., formal, startup, fast-paced).

---

##  2. CUSTOMIZE THE CANDIDATE’S CV

Using the job description and the candidate's original CV, tailor the resume to increase their chances of success.

**DO:**
- Emphasize relevant experience, tools, technologies, or achievements.
- Reorder sections if necessary to align with job expectations.
- Improve clarity, formatting, and impact (e.g., use action verbs and quantifiable results).
- Adjust language and keywords to match the job post style.

**DO NOT:**
- Invent or fabricate any experience or qualification.
- Add details not present in the original CV.

**Maintain** a professional tone and resume format.

---

###  Return your output in **two clear sections**:

---

### 1. COMMENTS AND TAILORING STRATEGY
Explain:
- What key details were extracted from the job post.
- What changes were made to improve alignment.
- Any section you removed, emphasized, or rephrased — and why.

---

### 2. FINAL CUSTOMIZED CV
Return the tailored resume here.

---

 **Job Description**:
{job_description}

---

 **Original CV**:
{self.cv_text}
"""

    def customize_cv_for_job(self, job_url):
        self.start_chat()
        job_description = self.extract_job_description(job_url)
        if not job_description:
            print("⚠️ Failed to extract job description.")
            return ""

        prompt = self.build_prompt(job_description)
        print("🧠 Sending to Gemini for tailoring...")
        response = self.chat.send_message(prompt)

        print("✅ Done! Here's your tailored CV:")
        return response.candidates[0].content.parts[0].text


### Run

In [ ]:
job_url = input("🔗 Paste the job URL: ")
cv_customizer = CVCustomizerAgent(api_key=GOOGLE_API_KEY, cv_text=cv_text)
customized_cv = cv_customizer.customize_cv_for_job(job_url)

Markdown(textwrap.dedent(customized_cv))